In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img ,img_to_array
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from keras.layers import Conv2D,Dropout,MaxPooling2D,Flatten,Dense,BatchNormalization
from tensorflow.keras import regularizers
import keras
import shutil
import os
from keras_vggface.vggface import VGGFace
from IPython.display import clear_output
from sklearn.metrics import accuracy_score,confusion_matrix
import seaborn as sns
from keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.applications.resnet50 import preprocess_input
import tensorflow as tf


In [2]:
import tensorflow
print(tensorflow.__version__)


2.9.1


In [3]:
x = []
y = []
datadir = r'Brain MRI Images for Brain Tumor Detection'
Categories = []
for file_label in os.listdir(datadir):
    Categories.append(file_label)
    for file_name in os.listdir(datadir+'/'+file_label):
        x.append(datadir+'/'+file_label+'/'+file_name)
        y.append(file_label)

dataset=pd.DataFrame({
        'image_path' : x,
        'label' : y
    })

In [4]:
train_df, test_df = train_test_split(dataset, test_size=0.2, shuffle=True, random_state=123, stratify=dataset['label'])
#test_df, valid_df = train_test_split(test_df, test_size=0.5, shuffle=True, random_state=123)

In [5]:
print("train_df = ",len(train_df))
#print("valid_df = ",len(valid_df))
print("test_df = ",len(test_df))

train_df =  202
test_df =  51


In [6]:
train_datagen = ImageDataGenerator(horizontal_flip= True)

test_datagen = ImageDataGenerator()

In [7]:
training_set = train_datagen.flow_from_dataframe( train_df, 
                                       x_col= 'image_path', 
                                       y_col= 'label', 
                                       target_size= (224,224), 
                                       class_mode= 'categorical',
                                       color_mode= "rgb", 
                                       shuffle= True, 
                                       batch_size= 32)

Found 202 validated image filenames belonging to 2 classes.


In [8]:
vald_set = test_datagen.flow_from_dataframe( valid_df, 
                                       x_col= 'image_path', 
                                       y_col= 'label', 
                                       target_size= (224,224), 
                                       class_mode= 'categorical',
                                       color_mode= "rgb", 
                                       shuffle= True, 
                                       batch_size= 32)

NameError: name 'valid_df' is not defined

In [9]:
test_set = test_datagen.flow_from_dataframe( test_df, 
                                      x_col= 'image_path', 
                                      y_col= 'label', 
                                      target_size= (224,224), 
                                      class_mode= 'categorical',
                                      color_mode= "rgb", 
                                      shuffle= False, 
                                      batch_size= 32)

Found 51 validated image filenames belonging to 2 classes.


In [10]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)

In [16]:
base_model = keras.models.load_model('44class_96.5.h5')
for layer in base_model.layers:
    layer.trainable=False

In [17]:
base_model.pop()
base_model.pop()
base_model.pop()

In [18]:
base_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb5 (Functional)  (None, 2048)             28513527  
                                                                 
 batch_normalization (BatchN  (None, 2048)             8192      
 ormalization)                                                   
                                                                 
Total params: 28,521,719
Trainable params: 0
Non-trainable params: 28,521,719
_________________________________________________________________


In [19]:
model = Sequential([
    base_model,
    BatchNormalization(axis= -1, momentum= 0.99, epsilon= 0.001),
    Dense(256, 
          kernel_regularizer= regularizers.l2(l= 0.016), 
          activity_regularizer= regularizers.l1(0.006),
          bias_regularizer= regularizers.l1(0.006), 
          activation= 'relu'),
    
    Dropout(rate= 0.45, 
            seed= 123),
    Dense(256, activation= 'relu'),
    Dense(2, activation= 'softmax')
])

In [20]:
""" model = Sequential()
model.add(base_model)
model.add(Dense(2, activation="softmax")) """

' model = Sequential()\nmodel.add(base_model)\nmodel.add(Dense(2, activation="softmax")) '

In [22]:
model.compile(Adamax(learning_rate= 0.0001), loss= 'categorical_crossentropy', metrics= ['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 2048)              28521719  
                                                                 
 batch_normalization_1 (Batc  (None, 2048)             8192      
 hNormalization)                                                 
                                                                 
 dense_3 (Dense)             (None, 256)               524544    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_4 (Dense)             (None, 256)               65792     
                                                                 
 dense_5 (Dense)             (None, 2)                 514       
                                                      

In [23]:
early_stopping = EarlyStopping(monitor="val_loss", patience=10)

checkpoint_name = 'Best-val_accuracy.h5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_accuracy', verbose = 1, save_best_only = True,save_weights_only=True, mode ='auto')

checkpoint_name1 = 'Best-val_loss.h5' 
checkpoint1 = ModelCheckpoint(checkpoint_name1, monitor='val_loss', verbose = 1, save_best_only = True,save_weights_only=True, mode ='auto')

In [24]:
res = model.fit(training_set,validation_data = test_set, epochs=300,shuffle=False,callbacks=[checkpoint,checkpoint1,early_stopping])

Epoch 1/300
7/7 [==============================] - ETA: 0s - loss: 8.8816 - accuracy: 0.5990
Epoch 1: val_accuracy improved from -inf to 0.78431, saving model to Best-val_accuracy.h5

Epoch 1: val_loss improved from inf to 8.49690, saving model to Best-val_loss.h5
7/7 [==============================] - 25s 2s/step - loss: 8.8816 - accuracy: 0.5990 - val_loss: 8.4969 - val_accuracy: 0.7843
Epoch 2/300
7/7 [==============================] - ETA: 0s - loss: 8.7586 - accuracy: 0.6436
Epoch 2: val_accuracy improved from 0.78431 to 0.84314, saving model to Best-val_accuracy.h5

Epoch 2: val_loss improved from 8.49690 to 8.40173, saving model to Best-val_loss.h5
7/7 [==============================] - 4s 675ms/step - loss: 8.7586 - accuracy: 0.6436 - val_loss: 8.4017 - val_accuracy: 0.8431
Epoch 3/300
7/7 [==============================] - ETA: 0s - loss: 8.5743 - accuracy: 0.7079
Epoch 3: val_accuracy improved from 0.84314 to 0.86275, saving model to Best-val_accuracy.h5

Epoch 3: val_loss im

In [28]:
model.load_weights('Best-val_accuracy.h5')
#model = keras.models.load_model('Weights-038-val_loss-0.43673.h5')

In [12]:
train_score = model.evaluate(training_set)
#valid_score = model.evaluate(vald_set)
test_score = model.evaluate(test_set)

print("Train Loss: ", train_score[0])
print("Train Accuracy: ", train_score[1])
print('-' * 20)
#print("Validation Loss: ", valid_score[0])
#print("Validation Accuracy: ", valid_score[1])
print('-' * 20)
print("Test Loss: ", test_score[0])
print("Test Accuracy: ", test_score[1])

2/2 [==============================] - 2s 1s/step - loss: 0.0932 - accuracy: 1.0000
Train Loss:  0.10673811286687851
Train Accuracy:  0.9851484894752502
--------------------
--------------------
Test Loss:  0.09316087514162064
Test Accuracy:  1.0


In [29]:
train_score = model.evaluate(training_set)
#valid_score = model.evaluate(vald_set)
test_score = model.evaluate(test_set)

print("Train Loss: ", train_score[0])
print("Train Accuracy: ", train_score[1])
print('-' * 20)
#print("Validation Loss: ", valid_score[0])
#print("Validation Accuracy: ", valid_score[1])
print('-' * 20)
print("Test Loss: ", test_score[0])
print("Test Accuracy: ", test_score[1])

2/2 [==============================] - 0s 137ms/step - loss: 6.7520 - accuracy: 0.9804
Train Loss:  6.626478672027588
Train Accuracy:  1.0
--------------------
--------------------
Test Loss:  6.752020835876465
Test Accuracy:  0.9803921580314636


In [ ]:
#model.save('2class_98.0.h5')

In [11]:
model = keras.models.load_model('2calss_lagre_dataset_99.1.h5')

In [ ]:
img = tf.keras.utils.load_img(
    "0a0bc6879f5d5d14c4df229b64b801_big_gallery.jpeg",
    grayscale=False,
    color_mode='rgb',
    target_size=(224,224),
    interpolation='nearest',
    keep_aspect_ratio=False
    )
plt.imshow(img)
plt.axis('off')


In [ ]:
img = np.array(img).reshape(-1, 224, 224, 3)
img.shape

In [ ]:
y_pred = model.predict(img)
y_pred_labels = np.argmax(y_pred, axis=1)

In [ ]:
result=['Astrocitoma T1',
 'Astrocitoma T1C+',
 'Astrocitoma T2',
 'Carcinoma T1',
 'Carcinoma T1C+',
 'Carcinoma T2',
 'Ependimoma T1',
 'Ependimoma T1C+',
 'Ependimoma T2',
 'Ganglioglioma T1',
 'Ganglioglioma T1C+',
 'Ganglioglioma T2',
 'Germinoma T1',
 'Germinoma T1C+',
 'Germinoma T2',
 'Glioblastoma T1',
 'Glioblastoma T1C+',
 'Glioblastoma T2',
 'Granuloma T1',
 'Granuloma T1C+',
 'Granuloma T2',
 'Meduloblastoma T1',
 'Meduloblastoma T1C+',
 'Meduloblastoma T2',
 'Meningioma T1',
 'Meningioma T1C+',
 'Meningioma T2',
 'Neurocitoma T1',
 'Neurocitoma T1C+',
 'Neurocitoma T2',
 'Oligodendroglioma T1',
 'Oligodendroglioma T1C+',
 'Oligodendroglioma T2',
 'Papiloma T1',
 'Papiloma T1C+',
 'Papiloma T2',
 'Schwannoma T1',
 'Schwannoma T1C+',
 'Schwannoma T2',
 'Tuberculoma T1',
 'Tuberculoma T1C+',
 'Tuberculoma T2',
 '_NORMAL T1',
 '_NORMAL T2']

In [ ]:
result[y_pred_labels[0]]

In [ ]:
plt.figure(figsize= (20, 8))
plt.style.use('fivethirtyeight')

plt.subplot(1, 2, 1)
plt.plot(res.history['loss'], 'r', label= 'Training loss')
plt.plot(res.history['val_loss'], 'b', label= 'Validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(res.history['accuracy'], 'r', label= 'Training Accuracy')
plt.plot(res.history['val_accuracy'], 'b', label= 'Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout
plt.savefig('EfficientNetB5_2calss.png')
plt.show()